![alt text](../img/header.jpg)

# Flopy Tips and Tricks for MODFLOW 6 Models

* Simplest running model
* Working with arrays
* Working with lists


In [ ]:
import os
import sys
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

## Simplest Running Model

In [ ]:
model_ws = './ex0x-completed-tricks'

In [ ]:
sim = flopy.mf6.MFSimulation(exe_name=config.mf6exe, 
                             sim_ws=model_ws)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)
ims = flopy.mf6.modflow.mfims.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim)
dis = flopy.mf6.ModflowGwfdis(gwf)
ic = flopy.mf6.ModflowGwfic(gwf)
npf = flopy.mf6.ModflowGwfnpf(gwf)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 0.]])
sim.write_simulation()
sim.run_simulation()

## Working with Arrays

### List of 2D Arrays

In [ ]:
sim = flopy.mf6.MFSimulation(exe_name=config.mf6exe, 
                             sim_ws=model_ws)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)
ims = flopy.mf6.modflow.mfims.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim)

# set up grid information
nlay, nrow, ncol = 3, 12, 10
top = 0.
dz = 1.
botm = [top - (k + 1) * dz for k in range(nlay)]
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              top=top, botm=botm)

ic = flopy.mf6.ModflowGwfic(gwf)

# pass in a list of two dimensional arrays
k1 = np.random.random((nrow, ncol))
k2 = np.random.random((nrow, ncol))
k3 = np.random.random((nrow, ncol))
npf = flopy.mf6.ModflowGwfnpf(gwf, k=[k1, k2, k3])

chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 0.],
                                                       [(0, nrow - 1, ncol - 1), 1.],])
sim.write_simulation()
sim.run_simulation()

### Using an External Array File

In [ ]:
sim = flopy.mf6.MFSimulation(exe_name=config.mf6exe, 
                             sim_ws=model_ws)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)
ims = flopy.mf6.modflow.mfims.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim)

# set up grid information
nlay, nrow, ncol = 3, 12, 10
top = 0.
dz = 1.
botm = [top - (k + 1) * dz for k in range(nlay)]
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              top=top, botm=botm)

ic = flopy.mf6.ModflowGwfic(gwf)

# pass in a list with one layer passed as a dictionary
k1 = np.random.random((nrow, ncol))
k2a = np.random.random((nrow, ncol))
k2 = {'filename': 'model.npf.k1.dat', 'data': k2a, 'factor':2.0}
k3 = np.random.random((nrow, ncol))
npf = flopy.mf6.ModflowGwfnpf(gwf, k=[k1, k2, k3])

chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 0.],
                                                       [(0, nrow - 1, ncol - 1), 1.],])
sim.write_simulation()
sim.run_simulation()

### Using an External Binary Array File

In [ ]:
def write_binary_file(fname, arr):
    text = 'head'
    pertim = np.float64(1.0)
    nrow = arr.shape[0]
    ncol = arr.shape[1]
    precision = 'double'
    header = flopy.utils.BinaryHeader.create(bintype=text, precision=precision,
                                             text=text, nrow=nrow, ncol=ncol,
                                             ilay=1, pertim=pertim,
                                             totim=pertim, kstp=1, kper=1)
    flopy.utils.Util2d.write_bin(arr.shape, fname, arr, header_data=header)
    return
    
sim = flopy.mf6.MFSimulation(exe_name=config.mf6exe, 
                             sim_ws=model_ws)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)
ims = flopy.mf6.modflow.mfims.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim)

# set up grid information
nlay, nrow, ncol = 3, 12, 10
top = 0.
dz = 1.
botm = [top - (k + 1) * dz for k in range(nlay)]
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              top=top, botm=botm)

ic = flopy.mf6.ModflowGwfic(gwf)

k1 = np.random.random((nrow, ncol))

# Layer 2 K from binary file
k2a = np.random.random((nrow, ncol))
fname = 'model.npf.k1.bin'
write_binary_file(os.path.join(model_ws, fname), k2a)
k2 = {'filename': fname, 'binary': True, 'iprn':1}

k3 = np.random.random((nrow, ncol))
npf = flopy.mf6.ModflowGwfnpf(gwf, k=[k1, k2, k3])

chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 0.],
                                                       [(0, nrow - 1, ncol - 1), 1.],])
sim.write_simulation()
sim.run_simulation()

## Working with Lists

### Internal Lists

In [ ]:
sim = flopy.mf6.MFSimulation(exe_name=config.mf6exe, 
                             sim_ws=model_ws)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)
ims = flopy.mf6.modflow.mfims.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim)

# set up grid information
nlay, nrow, ncol = 3, 12, 10
top = 0.
dz = 1.
botm = [top - (k + 1) * dz for k in range(nlay)]
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              top=top, botm=botm)
ic = flopy.mf6.ModflowGwfic(gwf)
npf = flopy.mf6.ModflowGwfnpf(gwf)

chd_spd = [[(0, i, 0), 0.] for i in range(nrow)]
chd_spd += [[(0, i, ncol - 1), 1.] for i in range(nrow)]
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)

sim.write_simulation()
sim.run_simulation()

### Using BOUNDNAMES

In [ ]:
sim = flopy.mf6.MFSimulation(exe_name=config.mf6exe, 
                             sim_ws=model_ws)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)
ims = flopy.mf6.modflow.mfims.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim)

# set up grid information
nlay, nrow, ncol = 3, 12, 10
top = 0.
dz = 1.
botm = [top - (k + 1) * dz for k in range(nlay)]
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              top=top, botm=botm)
ic = flopy.mf6.ModflowGwfic(gwf)
npf = flopy.mf6.ModflowGwfnpf(gwf)

chd_spd = [[(0, i, 0), 0., 'left'] for i in range(nrow)]
chd_spd += [[(0, i, ncol - 1), 1., 'right'] for i in range(nrow)]
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd, boundnames=True)

fname = 'model.chd.obs.dat'
chd_obs = {fname: [('left_flow', 'CHD', 'LEFT'),
                   ('right_flow', 'CHD', 'RIGHT')]}
chd.obs.initialize(continuous=chd_obs)

sim.write_simulation()
sim.run_simulation()

### External File Lists

In [ ]:
sim = flopy.mf6.MFSimulation(exe_name=config.mf6exe, 
                             sim_ws=model_ws)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)
ims = flopy.mf6.modflow.mfims.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim)

# set up grid information
nlay, nrow, ncol = 3, 12, 10
top = 0.
dz = 1.
botm = [top - (k + 1) * dz for k in range(nlay)]
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              top=top, botm=botm)
ic = flopy.mf6.ModflowGwfic(gwf)
npf = flopy.mf6.ModflowGwfnpf(gwf)

chd_spd = [[(0, i, 0), 0.] for i in range(nrow)]
chd_spd += [[(0, i, ncol - 1), 1.] for i in range(nrow)]
chd_spd0 = {'filename': 'model.chd.sp0', 'data': chd_spd}
chd_spd = {0: chd_spd0}
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd, maxbound=nrow*2)

sim.write_simulation()
sim.run_simulation()

### External Binary File Lists

In [ ]:
# not working in Flopy yet, but close...
sim = flopy.mf6.MFSimulation(exe_name=config.mf6exe, 
                             sim_ws=model_ws)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)
ims = flopy.mf6.modflow.mfims.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim)

# set up grid information
nlay, nrow, ncol = 3, 12, 10
top = 0.
dz = 1.
botm = [top - (k + 1) * dz for k in range(nlay)]
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                              top=top, botm=botm)
ic = flopy.mf6.ModflowGwfic(gwf)
npf = flopy.mf6.ModflowGwfnpf(gwf)

dt = [('layer', np.int32), ('row', np.int32), ('col', np.int32), ('head', np.float64)]
dt = np.dtype(dt)
a = [(0, i, 0, 0.) for i in range(nrow)]
a += [(0, i, ncol - 1, 1.) for i in range(nrow)]
a = np.array(a, dtype=dt)
a['layer'] += 1
a['row'] += 1
a['col'] += 1

fname = 'model.chd.sp0'
fout = open(os.path.join(model_ws, fname), 'wb')
a.tofile(fout)
fout.close()

chd_spd0 = {'filename': fname, 'factor': None, 'binary': True}
chd_spd = {0: chd_spd0}
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd, maxbound=nrow*2)

sim.write_simulation()
sim.run_simulation()